En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# **Challenge LATAM AIRLINES Data Engineer**

For this challenge a data has been given, the data is about tweets from Twitter, it comes as a JSON file wich has a size of 389Mb aprox. 

At first we have to set the file into a place it can be accessed and not in the repository, by being a large file. 

I uploaded it to `Google Cloud Storage`, inside a `bucket`. From here we'll refer to `Google Cloud Storage` as `GCS`.

The files at `GCS` has a path like `"gs://{bucket_name}/{blob.name}"`. Bucket name and file is reserved by me in my `GCS` and `env file`.

I created a `.env` file where i set my enviroment variables as
- `PROJECT_ID`: The Google Cloud Platform (`GCP`) project where the bucket of `GCS` is allowed.
- `BUCKET_NAME`: The name of the `GCS` bucket
- `GCP_CREDENTIALS`: The Json content of the key to the service account in `GCP` to log into `Google Cloud`.
- `BUCKET_FOLDER_ORIGIN`: The folder inside the bucket where data is stored.

I added a python file named `json_to_dataframe.py` where i make the connection to `GCS`. 

This file has modules to get the file path and get a dataframe from the given file.

So, now that my environment is set `I can start to code!`.

At first we will get the file path from our `cloud bucket`.

In [1]:
import json_to_dataframe as jtd 
import os
import pandas as pd
import cProfile
import pstats
from memory_profiler import profile

file_path = jtd.get_gcs_path()

Logged in to GCP / GCS client
BLOBS LISTED! ----------------------
[<Blob: latam-airlines, Data/, 1710462726242408>, <Blob: latam-airlines, Data/farmers-protest-tweets.json, 1710465820905781>]
The file name is 
The file name is farmers-protest-tweets.json
BUILDING FILE PATH! ----------------------


Now that we have the file path, we can download the file to work it offline. 

First we call from jtd the method download file, this method downloads the file into folder `src/files/`. If the file already exist in the folder, it doesn't get downloaded. The method returns the local path of the json file.

With the file downloaded, the local path file is used to read the file with `pandas` and save it to parquet format (best by compression of large data) if the parquet file doesn't exist.

Finally, build the parquet file path in local and save it in a variable.







In [2]:
directory_path = os.path.join(os.path.dirname(os.path.abspath("q1_memory.py")), "files")
local_json_file_path = jtd.download_file(file_path,directory_path)
df= pd.read_json(local_json_file_path, lines=True)
# Get the file name from the file path
file_name = os.path.basename(local_json_file_path)
# parquet file name
parquet_file_name = file_name.split('.')[0] + '.parquet'
# If parquet file does not exist, create a new parquet file
if not os.path.exists(os.path.join(directory_path, parquet_file_name)):
    df.to_parquet(os.path.join(directory_path, parquet_file_name),engine='auto')
    print("Parquet file created")
# Get the path of the parquet file
parquet_file_local_path = os.path.join(directory_path, parquet_file_name)
# Clear df from memory
del df

All questions receive as parameters a file_path and have two approaches, one **based on time** and the other one **based on memory**.

The time based approaches are optimal with the parquet files, by that reason we will pass as path parameter the parquet file path.



A better way to make better this comparison and questions is by using a ***database or data warehouse***, to execute querys. Personally, I use ***BigQuery*** because it computes querys in large remote cluster servers at the same time, which makes the time response short by dividing one Query in multiple sub-queries, optimizing memory and time at the same time. And can be consumed by reporting tools by connecting to data.

### ***Question/Challenge 1***

"Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días."

Top 10 dates where there are the most tweets. Mention the user (username) who has the most publications for each of those days.

For consulting this, we have to read the data from the file in json/parquet. 
#### **Time Approach**
I used pandas as my reader/interpreter, it returns a dataframe from the parquet file previously built.

In this module we read the parquet file using the pandas functionality .read_parquet, otherwise, i allowed to the method the possibility to read json file if json file is passed as parameter.
```python
if file_path.endswith(".json"):
    df = pd.read_json(file_path, lines=True)
elif file_path.endswith(".parquet"):
    df = pd.read_parquet(file_path)
```

This approach use pandas as the reader, it reads all the file and keep it as a dataframe in memory. That makes it better for time of response to the consult, but makes data is always loaded at memory.

At first, convert the date column from datetime to format date. Takes username from user as the user.

Use `.value_counts()` to count over date field of dataframe, counts all ocurrences in the field and order it from bigger to smaller, we take the first 10 with `.head(10)`
```python
top_10_dates = df["date"].value_counts().head(10)
```

Now we have to get the user with more tweets (rows or count) on each date, by iterating on each date and using date as dataframe filter, where the user field is used in value_counts to make a count by user on the specific day. After that, take `idxmax()` to get the first row, also can be used `.head(1)`, this takes the user with most tweets. User and date are append to the python list `result`. Result returns the 10 dates with the user with most tweets in each date.

```python
result = []
# Iterate over the top 10 dates
for date in top_10_dates.index:
    # Get the user with the most tweets in the date
    user = df[df["date"] == date]["user"].value_counts().idxmax()
    # Append the date and the user to the result list
    result.append((date, user))
```

The full code of this approach is at 
[q1_time.py](https://github.com/JoRgEXx1899/latam-de-challenge/blob/main/src/q1_time.py).

In [3]:
from q1_time import q1_time

q1_t_result = q1_time(parquet_file_local_path)
print("q1_time.py executed successfully")
print("Results:")
print(q1_t_result)

q1_time.py executed successfully
Results:
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


#### **Memory Approach**

For this approach pandas is not used pandas as the reader, because we have to optimize the memory use. Pandas, as i explained before, keeps data on memory which makes it not optimal for a memory optimizing approach. 


This approach iterates line by line of the JSON file, ensuring one line at time in memory.

Why not to use the parquet file? Because parquet file is focused on making better execution throughout the time, but not memory.

Uses a dictionary to store by date the user count. Line by line gets the date and user of the tweet and adds +1 to the count of the user in the date.

With the dictionary built, just have to get the top user by date, then append date and top user into a list. The list is sorted according with most date count from major to minor. We slice the first 10 lines of the list and return it as the result. The full code of this approach is at [q1_memory.py](https://github.com/JoRgEXx1899/latam-de-challenge/blob/main/src/q1_memory.py).

In [4]:
from q1_memory import q1_memory

q1_m_result = q1_memory(local_json_file_path)
print("q1_memory.py executed successfully")
print("Results:")
print(q1_m_result)

q1_memory.py executed successfully
Results:
[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 19), 'Preetm91'), (datetime.date(2021, 2, 21), 'Surrypuria'), (datetime.date(2021, 2, 23), 'Surrypuria')]


Now that we have the result and code. Make some measures about both approaches. At first we evaluate memory approach.

In [5]:
# Load the memory_profiler extension to evaluate the memory usage in the functions
%load_ext memory_profiler
%reload_ext memory_profiler

# Evaluates the memory usage of the q1_time function
print("Memory Evaluation q1_time\n")
%memit q1_time(parquet_file_local_path)

# Evaluates the memory usage of the q1_memory function
print("\nMemory Evaluation q1_memory\n")
%memit q1_memory(local_json_file_path)


Memory Evaluation q1_time

peak memory: 1715.60 MiB, increment: 859.63 MiB

Evaluación de memoria q1_memory

Memory Evaluation q1_memory

peak memory: 1252.55 MiB, increment: 2.66 MiB


As we can see, the memory used by q1_memory is smaller than the memory used by the q1_time approach.

Now the turn for time approach. First we evaluate `q1_time` function.

In [6]:
# Instantiate the profiler
profiler = cProfile.Profile()
# Run the profiler for q1_time function
profiler.runcall(q1_time, parquet_file_local_path)
# Print the statistics of the profiler
# Get the total time spent in the function
#profiler.print_stats(sort='cumtime')
# Disable the cProfile profiler and save statistics to a file
profiler.disable()
profiler.dump_stats("output.pstats1")
# Create an instance of pstats Stats to analyze the statistics
stats = pstats.Stats("output.pstats1")
# Sort the statistics by the time spent in the function
stats.sort_stats("cumtime")
# Print the statistics
stats.print_stats(2)


Fri Mar 15 18:50:34 2024    output.pstats1

         134733 function calls (134410 primitive calls) in 1.909 seconds

   Ordered by: cumulative time
   List reduced from 622 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    1.909    1.909 c:\Users\danie\OneDrive\Documents\GitHub\latam-de-challenge\src\q1_time.py:7(q1_time)
        1    0.002    0.002    1.789    1.789 c:\Users\danie\.conda\envs\latamair\Lib\site-packages\pandas\io\parquet.py:428(read_parquet)




And now evaluate `q1_memory` function.

In [7]:
# Run the profiler for q1_time function
profiler.runcall(q1_memory, local_json_file_path)
# Print the statistics of the profiler
# Get the total time spent in the function
#profiler.print_stats(sort='cumtime')
# Disable the cProfile profiler and save statistics to a file
profiler.disable()
profiler.dump_stats("output.pstats1")
# Create an instance of pstats Stats to analyze the statistics
stats = pstats.Stats("output.pstats1")
# Sort the statistics by the time spent in the function
stats.sort_stats("cumtime")
# Print the statistics
stats.print_stats(2)

# Remove the pstats file
os.remove("output.pstats1")

Fri Mar 15 18:50:39 2024    output.pstats1

         3404336 function calls (3404013 primitive calls) in 6.538 seconds

   Ordered by: cumulative time
   List reduced from 647 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.994    0.994    4.630    4.630 c:\Users\danie\OneDrive\Documents\GitHub\latam-de-challenge\src\q1_memory.py:7(q1_memory)
        1    0.002    0.002    1.909    1.909 c:\Users\danie\OneDrive\Documents\GitHub\latam-de-challenge\src\q1_time.py:7(q1_time)




As wee can see, the `q1_time` function is better in execution time than `q1_memory` function.

### ***Question/Challenge 2***

"Los top 10 emojis más usados con su respectivo conteo

"The top 10 most used emojis with their respective counts."

For consulting this, we have to read the data from the file in json/parquet. Also, to find emojis easily we'll use `emoji` library.
#### **Time Approach**
I used pandas as my reader/interpreter, it returns a dataframe from the parquet file previously built.

In this module we read the parquet file using the pandas functionality .read_parquet, otherwise, i allowed the method the possibility to read json file if json file is passed as parameter.
```python
if file_path.endswith(".json"):
    df = pd.read_json(file_path, lines=True)
elif file_path.endswith(".parquet"):
    df = pd.read_parquet(file_path)
```

This approach use pandas as the reader, it reads all the file and keep it as a dataframe in memory. That makes it better for time of response to the consult, but makes data is always loaded at memory.

At first, create a emoji_list to store all the emojis find in content of the tweets. Then iterates over `df["content"]`. By each iteration a list is created with the emojis in the tweet, to do this emoji library is used to create a list with emojis found in the contentm then gets the emoji of the list and store it in a list, finally in the iteration emojis are pushed or extended to the emoji_list.

```python
# Iterates through the 'content' column of the DataFrame
for content in df['content']:
    # Extract the emojis from each content using the emoji package
    emojis_in_tweet = [
        emoticon['emoji'] for emoticon in emoji.emoji_list(content)
    ]
    # Append the emojis to the list
    emoji_list.extend(emojis_in_tweet)
```

Use Counter from collections library to make a frequency dictionary of the emojis (unique emojis and it's count). Then the dictionary is sorted by value and sliced to get the top 10 used emojis.
```python
# Count the frequency of each emoji in the list
emoji_freq = Counter(emoji_list)
# Sort the dictionary by value in descending order
sorted_emoji_freq = sorted(
    emoji_freq.items(), key=lambda x: x[1], reverse=True)[:10]
return sorted_emoji_freq
```

The full code of this approach is at 
[q2_time.py](https://github.com/JoRgEXx1899/latam-de-challenge/blob/main/src/q2_time.py).

In [8]:
from q2_time import q2_time

q2_t_result = q2_time(parquet_file_local_path)
print("q2_time.py executed successfully")
print("Results:")
print(q2_t_result)

q2_time.py executed successfully
Results:
[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('🤣', 1668), ('✊', 1651), ('❤️', 1382), ('🙏🏻', 1317), ('💚', 1040)]


#### **Memory Approach**

For this approach pandas is not used pandas as the reader, because we have to optimize the memory use. Pandas, as i explained before, keeps data on memory which makes it not optimal for a memory optimizing approach. 

This approach iterates line by line of the JSON file, ensuring one line at time in memory.

Why not to use the parquet file? Because parquet file is focused on making better execution throughout the time, but not memory.

Uses a dictionary to store the emoji count. Line by line gets the emojis in the content of the tweet by using emoji library. Builds a list of emojis in the line and adds +1 to the count of each emoji in the list. Then, iterates on the next line.

```python
# Get the content of the tweet or an empty string if there is no content
contenido = tweet.get('content', '')
# Extract the emojis from the tweet content and update the emoji counter
emojis_in_tweet = [emoticon['emoji']
                    for emoticon in emoji.emoji_list(contenido)]
for emojii in emojis_in_tweet:
    emoji_count[emojii] += 1
```
With the dictionary built, just have to get the top emojis by count, to do that we have to sort with `sorted()` method the dictionary in reverse order. We slice the first 10 lines of the dictionary and return it as the result. 

```python
result = sorted(
        emoji_count.items(), key=lambda x: x[1], reverse=True)[:10]
```

The full code of this approach is at [q2_memory.py](https://github.com/JoRgEXx1899/latam-de-challenge/blob/main/src/q2_memory.py).

In [9]:
from q2_memory import q2_memory

q2_m_result = q2_memory(local_json_file_path)
print("q2_memory.py executed successfully")
print("Results:")
print(q2_m_result)

q2_memory.py executed successfully
Results:
[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('🤣', 1668), ('✊', 1651), ('❤️', 1382), ('🙏🏻', 1317), ('💚', 1040)]


Now that we have the result and code. Make some measures about both approaches. At first we evaluate memory approach.

In [10]:
# Load the memory_profiler extension to evaluate the memory usage in the functions
%load_ext memory_profiler
%reload_ext memory_profiler

# Evaluates the memory usage of the q2_time function
print("Memory Evaluation q2_time\n")
%memit q2_time(parquet_file_local_path)

# Evaluates the memory usage of the q2_memory function
print("\nMemory Evaluation q2_memory\n")
%memit q2_memory(local_json_file_path)

Memory Evaluation q2_time

peak memory: 2280.62 MiB, increment: 653.39 MiB

Memory Evaluation q2_memory

peak memory: 1823.74 MiB, increment: 0.66 MiB


As we can see, the memory used by q2_memory is smaller than the memory used by the q2_time approach.

Now the turn for time approach. First we evaluate `q2_time` function.

In [14]:
# Instantiate the profiler
profiler = cProfile.Profile()
# Run the profiler for q2_time function
profiler.runcall(q2_time, parquet_file_local_path)
# Disable the cProfile profiler and save statistics to a file
profiler.disable()
profiler.dump_stats("output.pstats1")
# Create an instance of pstats Stats to analyze the statistics
stats = pstats.Stats("output.pstats1")
# Sort the statistics by the time spent in the function
stats.sort_stats("cumtime")
# Print the statistics
stats.print_stats(2)
# Remove the pstats file
os.remove("output.pstats1")

Fri Mar 15 19:33:36 2024    output.pstats1

         68960891 function calls (68960875 primitive calls) in 24.806 seconds

   Ordered by: cumulative time
   List reduced from 414 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.107    0.107   27.763   27.763 c:\Users\danie\OneDrive\Documents\GitHub\latam-de-challenge\src\q2_time.py:8(q2_time)
   117407    0.060    0.000   25.774    0.000 c:\Users\danie\.conda\envs\latamair\Lib\site-packages\emoji\core.py:282(emoji_list)




And now evaluate `q2_memory` function.

In [15]:
# Instantiate the profiler
profiler = cProfile.Profile()
# Run the profiler for q2_memory function
profiler.runcall(q2_memory, local_json_file_path)
# Disable the cProfile profiler and save statistics to a file
profiler.disable()
profiler.dump_stats("output.pstats1")
# Create an instance of pstats Stats to analyze the statistics
stats = pstats.Stats("output.pstats1")
# Sort the statistics by the time spent in the function
stats.sort_stats("cumtime")
# Print the statistics
stats.print_stats(2)
# Remove the pstats file
os.remove("output.pstats1")

Fri Mar 15 19:36:14 2024    output.pstats1

         86199109 function calls in 28.658 seconds

   Ordered by: cumulative time
   List reduced from 23 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.953    0.953   28.658   28.658 c:\Users\danie\OneDrive\Documents\GitHub\latam-de-challenge\src\q2_memory.py:7(q2_memory)
   117407    0.069    0.000   25.615    0.000 c:\Users\danie\.conda\envs\latamair\Lib\site-packages\emoji\core.py:282(emoji_list)




As wee can see, the `q2_time` function is better in execution time than `q2_memory` function.

### ***Question/Challenge 3***

"El top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos"

The historical top 10 most influential users (usernames) based on the count of mentions (@) each of them registers.
For consulting this, we have to read the data from the file in json/parquet. In this challenge we have to use the library `re` to work with regular expressions.

#### **Time Approach**
I used pandas as my reader/interpreter, it returns a dataframe from the parquet file previously built.

In this module we read the parquet file using the pandas functionality .read_parquet, otherwise, i allowed to the method the possibility to read json file if json file is passed as parameter.
```python
if file_path.endswith(".json"):
    df = pd.read_json(file_path, lines=True)
elif file_path.endswith(".parquet"):
    df = pd.read_parquet(file_path)
```

This approach use pandas as the reader, it reads all the file and keep it as a dataframe in memory. That makes it better for time of response to the consult, but makes data is always loaded at memory.

At first a list of mentioned users is created. Then iterates throug the content and extract all occurrences of @ followed of 1 or more characters, interpreting these as mentioned usernames. The mentioned username get pushed into the mentioned users list. 

```python
# Creates an empty list to store all the mentioned users in the tweets contents
mentioned_user_list = []
# Iterates through the 'content' column of the DataFrame
for content in df['content']:
    # Extract all the mentions r'@(\w+)' from content
    mentions_in_tweet = [
        mention for mention in re.findall(r'@(\w+)', content)
    ]
    # Append the user to the mentioned users list
    mentioned_user_list.extend(mentions_in_tweet)
```

With the list of mentioned users a counter is done to get frequency of of each user in the list. Then the list is sorted with `sorted()` in reverse (major to minor) and sliced to get the top 10 mentioned users.

``` python
# Count the frequency of each mentioned user in the list
    user_freq = Counter(mentioned_user_list)
    top_10_users=sorted(
        user_freq.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_10_users
```

The full code of this approach is at 
[q3_time.py](https://github.com/JoRgEXx1899/latam-de-challenge/blob/main/src/q3_time.py).

In [5]:
from q3_time import q3_time

q3_t_result = q3_time(parquet_file_local_path)
print("q3_time.py executed successfully")
print("Results:")
print(q3_t_result)

q3_time.py executed successfully
Results:
[('narendramodi', 2261), ('Kisanektamorcha', 1836), ('RakeshTikaitBKU', 1639), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1015), ('rihanna', 972), ('UNHumanRights', 962), ('meenaharris', 925)]


#### **Memory Approach**

For this approach pandas is not used pandas as the reader, because we have to optimize the memory use. Pandas, as i explained before, keeps data on memory which makes it not optimal for a memory optimizing approach. 

This approach iterates line by line of the JSON file, ensuring one line at time in memory.

Why not to use the parquet file? Because parquet file is focused on making better execution throughout the time, but not memory.

Uses a dictionary to store the mentioned users count. Line by line gets the mentions in the content of the tweet by using `re` library in code `re.findall(r'@(\w+)', content)`. Builds a list of mentioned users in the line and adds +1 to the count of each mentioned user in the list. Then, iterates on the next line.

```python
# Get the content of the tweet or an empty string if there is no content
content = tweet.get('content', '')
# Extract all the mentions r'@(\w+)' from content
mentions_in_tweet = [
    mention for mention in re.findall(r'@(\w+)', content)]
for mention in mentions_in_tweet:
    mentioned_users_count[mention] += 1
```
With the dictionary built, just have to get the top mentioned users by count, to do that we have to sort with `sorted()` method the dictionary in reverse order. We slice the first 10 lines of the dictionary and return it as the result. 

```python
result = sorted(
        emoji_count.items(), key=lambda x: x[1], reverse=True)[:10]
```

The full code of this approach is at [q3_memory.py](https://github.com/JoRgEXx1899/latam-de-challenge/blob/main/src/q3_memory.py).

In [6]:
from q3_memory import q3_memory

q3_m_result = q3_memory(local_json_file_path)
print("q3_memory.py executed successfully")
print("Results:")
print(q3_m_result)

q3_memory.py executed successfully
Results:
[('narendramodi', 2261), ('Kisanektamorcha', 1836), ('RakeshTikaitBKU', 1639), ('PMOIndia', 1422), ('RahulGandhi', 1125), ('GretaThunberg', 1046), ('RaviSinghKA', 1015), ('rihanna', 972), ('UNHumanRights', 962), ('meenaharris', 925)]


Now that we have the result and code. Make some measures about both approaches. At first we evaluate memory approach.

In [7]:
# Load the memory_profiler extension to evaluate the memory usage in the functions
%load_ext memory_profiler
%reload_ext memory_profiler

# Evaluates the memory usage of the q3_time function
print("Memory Evaluation q3_time\n")
%memit q3_time(parquet_file_local_path)

# Evaluates the memory usage of the q3_memory function
print("\nMemory Evaluation q3_memory\n")
%memit q3_memory(local_json_file_path)

Memory Evaluation q3_time

peak memory: 1942.26 MiB, increment: 761.30 MiB

Memory Evaluation q3_memory

peak memory: 1477.86 MiB, increment: 1.34 MiB


As we can see, the memory used by q3_memory is smaller than the memory used by the q3_time approach.

Now the turn for time approach. First we evaluate `q3_time` function.

In [8]:
# Instantiate the profiler
profiler = cProfile.Profile()
# Run the profiler for q3_time function
profiler.runcall(q3_time, parquet_file_local_path)
# Disable the cProfile profiler and save statistics to a file
profiler.disable()
profiler.dump_stats("output.pstats1")
# Create an instance of pstats Stats to analyze the statistics
stats = pstats.Stats("output.pstats1")
# Sort the statistics by the time spent in the function
stats.sort_stats("cumtime")
# Print the statistics
stats.print_stats(2)
# Remove the pstats file
os.remove("output.pstats1")

Fri Mar 15 21:49:20 2024    output.pstats1

         840473 function calls (840440 primitive calls) in 2.349 seconds

   Ordered by: cumulative time
   List reduced from 484 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.082    0.082    2.349    2.349 c:\Users\danie\OneDrive\Documents\GitHub\latam-de-challenge\src\q3_time.py:8(q3_time)
        1    0.002    0.002    2.023    2.023 c:\Users\danie\.conda\envs\latamair\Lib\site-packages\pandas\io\parquet.py:428(read_parquet)




And now evaluate `q3_memory` function.

In [9]:
# Instantiate the profiler
profiler = cProfile.Profile()
# Run the profiler for q3_memory function
profiler.runcall(q3_memory, local_json_file_path)
# Disable the cProfile profiler and save statistics to a file
profiler.disable()
profiler.dump_stats("output.pstats1")
# Create an instance of pstats Stats to analyze the statistics
stats = pstats.Stats("output.pstats1")
# Sort the statistics by the time spent in the function
stats.sort_stats("cumtime")
# Print the statistics
stats.print_stats(2)
# Remove the pstats file
os.remove("output.pstats1")

Fri Mar 15 21:49:35 2024    output.pstats1

         937436 function calls in 2.938 seconds

   Ordered by: cumulative time
   List reduced from 24 to 2 due to restriction <2>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.828    0.828    2.938    2.938 c:\Users\danie\OneDrive\Documents\GitHub\latam-de-challenge\src\q3_memory.py:8(q3_memory)
   117407    1.831    0.000    1.831    0.000 {built-in method ujson.loads}




As wee can see, the `q3_time` function is better in execution time than `q3_memory` function.